In [182]:
# import libraries
import matplotlib.pyplot as plt
from tensorflow import keras
import sklearn.metrics
import tensorflow as tf
import numpy as np
import sklearn
import pathlib
import shutil
import os

# create directories
BASE_MODEL = pathlib.Path('Models','base_model.keras')
MODEL = pathlib.Path('Models', 'digit_recognizer.keras')
LABELLED = pathlib.Path('Models', 'Dataset', 'Digit Recognizer', 'Labelled')

In [183]:
# load the base model
model = keras.models.load_model(BASE_MODEL)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics='accuracy'
)

In [184]:
# verify that base model is loaded properly
def transform(img):
    return (255 - img).astype('float32') / 255

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
transformed_x_train = transform(x_train)
transformed_x_test = transform(x_test)
encoder = sklearn.preprocessing.LabelBinarizer()
transformed_y_train = encoder.fit_transform(y_train)
transformed_y_test = encoder.transform(y_test)
model.evaluate(transformed_x_test, transformed_y_test)

In [186]:
# function to scale and one hot encode response
def transform(x, y):
    x = tf.cast(x, tf.float32) / 255
    x =  tf.image.resize(x, [28, 28])
    y = tf.one_hot(y, 10, dtype=tf.uint8)
    return x, y

In [187]:
# load the datasets
train_dataset = keras.utils.image_dataset_from_directory(
    LABELLED,
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(28, 28),
    shuffle=True,
    seed=1,
    validation_split=0.1,
    subset='training'
)

test_dataset = keras.utils.image_dataset_from_directory(
    LABELLED,
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(28, 28),
    shuffle=True,
    seed=1,
    validation_split=0.1,
    subset='validation'
)

train_dataset = train_dataset.map(transform)
test_dataset = test_dataset.map(transform)

Found 422 files belonging to 10 classes.
Using 380 files for training.
Found 422 files belonging to 10 classes.
Using 42 files for validation.


In [188]:
# train the model
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

Epoch 1/3
12/12 [==============================] - 2s 55ms/step - loss: 0.8569 - accuracy: 0.7868 - val_loss: 0.0575 - val_accuracy: 0.9762
Epoch 2/3
12/12 [==============================] - 1s 41ms/step - loss: 0.2861 - accuracy: 0.9132 - val_loss: 0.0132 - val_accuracy: 1.0000
Epoch 3/3
12/12 [==============================] - 1s 40ms/step - loss: 0.2046 - accuracy: 0.9395 - val_loss: 0.0099 - val_accuracy: 1.0000


In [189]:
# evaluate the model
model.evaluate(test_dataset)

2/2 [==============================] - 0s 11ms/step - loss: 0.0099 - accuracy: 1.0000


[0.009906304068863392, 1.0]

In [190]:
# extract dataset
images, labels = zip(*test_dataset)
labels = np.concatenate([l.numpy() for l in labels], axis=0)
labels = labels.argmax(axis=1)

In [191]:
# make predictions
predictions = []
for batch in images:
    predictions = np.concatenate([predictions, tf.argmax(model.predict(batch), axis=1).numpy()], axis=0)

1/1 [==============================] - 0s 114ms/step


1/1 [==============================] - 0s 108ms/step


In [192]:
# evaluate the model
print(
    sklearn.metrics.classification_report(
        labels,
        predictions,
        target_names=[str(x) for x in range(10)]
    )
)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00         5
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         3
           5       1.00      1.00      1.00         6
           6       1.00      1.00      1.00         4
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         6

    accuracy                           1.00        42
   macro avg       1.00      1.00      1.00        42
weighted avg       1.00      1.00      1.00        42



In [193]:
# save the model
model.save('digit_recognizer.keras')